os: Ubuntu
Version：Python 3
Website：www.douban.com
pip3 installl: beautifulsoup4, mysql.connector, lxml
apt-get install: python3-lxml

通过网站归类标签中“个人管理”类，获取书籍基础信息

In [2]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
from requests.utils import requote_uri


url='http://www.douban.com/tag/' + requote_uri('个人管理') + '/book?start=0'
html = requests.get(url)
soup = BeautifulSoup(html.content,'lxml')

class_books = soup.find('div', {'class': 'mod book-list'})
books = class_books.find_all('dd')
for book_info in books:
    title = book_info.find('a', {'class':'title'})
    print(title.get_text())
    desc = book_info.find('div', {'class':'desc'}).get_text()
    print(desc.strip())
    break

少有人走的路
[美] M·斯科特·派克 / 于海生 / 吉林文史出版社 / 2007-1 / 26.00元


统计“小说”标签类的书籍总量

In [4]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib


#Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]


page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '新书'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'

    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break

    page_number += 1
    try_times = 0
    
    
print('书籍数量：', page_number*20)
    

书籍数量： 1000


**Create database 'douban-books'**
```
CREATE DATABASE IF NOT EXISTS douban-books DEFAULT CHARSET = utf8 COLLATE = utf8_general_ci;
```

**Create table 'tag-novel'**
```
USE douban-books;

CREATE TABLE IF NOT EXISTS `tag-novel`(
   `novel_title` VARCHAR(100) NOT NULL,
   `novel_author` VARCHAR(20),
   `novel_publishing` VARCHAR(100),
   `novel_press_date` DATE,
   `novel_price` VARCHAR(100),
   `novel_img_website` VARCHAR(500),
   PRIMARY KEY ( `novel_title` )
)ENGINE=InnoDB DEFAULT CHARSET=utf8;
```

数据库连接

In [6]:
import mysql.connector as mariadb

mariadb_connection = mariadb.connect(
    host = 'localhost',
    user = 'root',
    password = '',
    database = 'douban_books'
)

cursor = mariadb_connection.cursor()

print(mariadb_connection)

插入标签“小说”类书籍信息到MySQL

In [ ]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib
import mysql.connector as mariadb


# Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]

# open a connection of mariadb
mariadb_connection = mariadb.connect(
    host = 'localhost',
    user = 'root',
    password = '',
    database = 'douban_books'
)

# get mariadb cursor
cursor = mariadb_connection.cursor()

page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '新书'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'
    
    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break
    # end if 
        
    for book_info in books:
        info = book_info.find('div', {'class': 'info'})
        title = info.find('a')['title'].strip()
        pub = info.find('div', {'class': 'pub'}).get_text().strip()
        list_pub = pub.split('/')


        if len(list_pub) == 4:
            author = list_pub[0].strip()
            publishing = list_pub[1].strip()
            press_year = list_pub[2].strip()
            price = list_pub[3].strip()
        # end if

        img = book_info.find('img')['src']
        # run error
        #preface = book_info.find('p').get_text().strip()
        
        # insert information to mariadb
        try:
            cursor.execute(
                "INSERT INTO tag_novel ( \
                    novel_title, \
                    novel_author, \
                    novel_publishing, \
                    novel_press_date, \
                    novel_price, \
                    novel_img_website \
                ) \
                VALUES (%s, %s, %s, %s, %s, %s)", 
                ( 
                    title,
                    author,
                    publishing,
                    press_year,
                    price,
                    img
                )
            )
        except mariadb.Error as error:
            print("Error: {}".format(error))
            continue
        # end try

    # end for

    page_number += 1
    try_times = 0
    
    print(page_number)
    
# end while

# mariadb transaction commit
mariadb_connection.commit()

# close a connection of mariadb
mariadb_connection.close()
    


For Example: 使用XML-RPC with python

In [4]:
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo

wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')
wp.call(GetPosts())

wp.call(GetUserInfo())

post = WordPressPost()
post.title = 'My new title'
post.content = 'This is the body of my new post.'
post.terms_names = {
    'post_tag': ['test', 'firstpost'],
    'category': ['Introductions', 'Tests']
}
wp.call(NewPost(post))

'5361'

带有特色图像缩略图的发布文章

In [4]:
import sys

sys.path

['C:\\Users\\xuzew\\Documents',
 'C:\\Users\\xuzew\\Anaconda3\\python37.zip',
 'C:\\Users\\xuzew\\Anaconda3\\DLLs',
 'C:\\Users\\xuzew\\Anaconda3\\lib',
 'C:\\Users\\xuzew\\Anaconda3',
 '',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\xuzew\\.ipython']

下载网上图片

In [9]:
import urllib.request

urllib.request.urlretrieve("https://img1.doubanio.com/view/subject/m/public/s28173928.jpg", "s28173928.jpg")


('s28173928.jpg', <http.client.HTTPMessage at 0x22c95a94f98>)

克隆豆瓣书籍英国小说下的书籍到Wordpress

In [ ]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib


# Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]

page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '小说'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'
    
    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break
    # end if 
        
    for book_info in books:
        info = book_info.find('div', {'class': 'info'})
        title = info.find('a')['title'].strip()
        pub = info.find('div', {'class': 'pub'}).get_text().strip()
        list_pub = pub.split('/')


        if len(list_pub) == 4:
            author = list_pub[0].strip()
            publishing = list_pub[1].strip()
            press_year = list_pub[2].strip()
            price = list_pub[3].strip()
        # end if

        img = book_info.find('img')['src']
        # run error
        #preface = book_info.find('p').get_text().strip()
        
        # insert information to mariadb
        try:
            cursor.execute(
                "INSERT INTO tag_novel ( \
                    novel_title, \
                    novel_author, \
                    novel_publishing, \
                    novel_press_date, \
                    novel_price, \
                    novel_img_website \
                ) \
                VALUES (%s, %s, %s, %s, %s, %s)", 
                ( 
                    title,
                    author,
                    publishing,
                    press_year,
                    price,
                    img
                )
            )
        except mariadb.Error as error:
            print("Error: {}".format(error))
            continue
        # end try

    # end for

    page_number += 1
    try_times = 0
    
    print(page_number)
    
# end while

    